In [6]:
import os

import torch
import torch.utils.data
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from PIL import Image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from src.engine import train_one_epoch, evaluate
from tqdm import tqdm

In [2]:
class COCODataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        coco_annotation = coco.loadAnns(ann_ids)
        path = coco.loadImgs(img_id)[0]['file_name']
        img = Image.open(os.path.join(self.root, path))

        num_objs = len(coco_annotation)

        boxes = []
        for i in range(num_objs):
            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            xmax = xmin + coco_annotation[i]['bbox'][2]
            ymax = ymin + coco_annotation[i]['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        class_labels = [annotation['category_id'] for annotation in coco_annotation]
        labels = torch.tensor(class_labels, dtype=torch.int64)

        img_id = torch.tensor([img_id])
        areas = []
        for i in range(num_objs):
            areas.append(coco_annotation[i]['area'])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)
    
def get_transform():
    transforms = []
    transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(transforms)

## DataLoad

In [3]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_data_dir = '../data/coco2017/train2017/'
train_coco = '../data/coco2017/annotations/instances_train2017.json'

train_batch_size = 2

my_dataset = COCODataset(root=train_data_dir,
                          annotation=train_coco,
                          transforms=get_transform()
                          )

data_loader = torch.utils.data.DataLoader(my_dataset,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=0,
                                          collate_fn=collate_fn,
                                          pin_memory=False)

test_data_dir = '../data/coco2017/val2017/'
test_coco = '../data/coco2017/annotations/instances_val2017.json'

test_coco = COCODataset(root=test_data_dir,
                          annotation=test_coco,
                          transforms=get_transform()
                          )

test_loader = torch.utils.data.DataLoader(test_coco,
                                          batch_size=2,
                                          shuffle=False,
                                          num_workers=0,
                                          collate_fn=collate_fn,
                                          pin_memory=False)

loading annotations into memory...
Done (t=20.22s)
creating index...
index created!
loading annotations into memory...
Done (t=0.67s)
creating index...
index created!


### Model

In [4]:
num_classes = 80

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
# checkpoint = torch.load('model/New_fasterRcnn_pidsix.pth')  # 가중치 파일의 경로를 지정하세요
# in_features = model.roi_heads.box_predictor.cls_score.in_features
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
# model.load_state_dict(checkpoint)

## Train

In [8]:
device = torch.device('cuda')

model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD( params, lr=0.0005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.01)

num_epochs = 10

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=1000)
    lr_scheduler.step()

    model_save_path = f'model/FasterRCNN_COCO2017{epoch}.pth'
    torch.save(model.state_dict(), model_save_path)

model_save_path = 'model/New_fasterRcnn_pidsix.pth'

torch.save(model.state_dict(), model_save_path)

print("That's it!")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## Test

In [9]:
device = torch.device('cuda')

num_classes = 14

print('Test Model Load')
test_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
in_features = test_model.roi_heads.box_predictor.cls_score.in_features
test_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
test_model.load_state_dict(torch.load('model/New_fasterRcnn_pidsix.pth'))
test_model.to(device)
test_model.eval()

print('------Prediction Start------')
all_predictions = []
for images, targets in tqdm(test_loader, desc='test',leave=True):
    images = [image.to(device) for image in images]
    with torch.no_grad():
        predictions = test_model(images)
    all_predictions.extend(predictions)

Test Model Load
------Prediction Start------


test: 100%|██████████| 5492/5492 [23:39<00:00,  3.87it/s]


In [10]:
coco_results = []

print('------Convert predictions and targets to COCO format------')
for prediction, target in zip(all_predictions, test_loader.dataset.coco.dataset['annotations']):
    for box, label, score in zip(prediction['boxes'], prediction['labels'], prediction['scores']):
        coco_result = {
            'image_id': target['image_id'],
            'category_id': label.item(),
            'bbox': box.cpu().numpy().tolist(),
            'score': score.item()
        }
        coco_results.append(coco_result)

coco_gt = COCO('json/My_test.json')
coco_dt = coco_gt.loadRes(coco_results)
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')

print('------Evaluate Start------')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

mAP_bbox = coco_eval.stats[0]
print("mAP_bbox:", mAP_bbox)

------Convert predictions and targets to COCO format------
loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!
------Evaluate Start------
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.61s).
Accumulating evaluation results...
DONE (t=1.80s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.088
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.082
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all |

In [16]:
import json

def convert_predictions_to_coco_format(predictions, image_ids):
    coco_results = []
    for prediction, image_id in zip(predictions, image_ids):
        for box, label, score in zip(prediction['boxes'], prediction['labels'], prediction['scores']):
            coco_result = {
                'image_id': image_id.item(),
                'category_id': label.item(),
                'bbox': box.cpu().numpy().tolist(),
                'score': score.item()
            }
            coco_results.append(coco_result)
    return coco_results

all_predictions = []
image_ids = []

print('------Prediction Start------')
for images, targets in tqdm(test_loader, desc='test', leave=True):
    images = [image.to(device) for image in images]
    image_ids.extend(targets[0]['image_id'].cpu().numpy())
    
    with torch.no_grad():
        predictions = test_model(images)
    all_predictions.extend(predictions)

# 예측 결과를 COCO 형식으로 변환
coco_results = convert_predictions_to_coco_format(all_predictions, image_ids)

# JSON 파일로 저장
output_json_path = 'predictions.json'
with open(output_json_path, 'w') as output_file:
    json.dump(coco_results, output_file, indent=2)

------Prediction Start------


test: 100%|██████████| 5492/5492 [23:50<00:00,  3.84it/s]


In [17]:
# 예측 결과를 COCO 형식으로 변환
coco_results = convert_predictions_to_coco_format(all_predictions, image_ids)

# JSON 파일로 저장
output_json_path = 'predictions.json'
with open(output_json_path, 'w') as output_file:
    json.dump(coco_results, output_file, indent=2)